In [1]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 34.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.8 MB/s eta 0:00:00


In [2]:
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 101.1 MB/s eta 0:00:0000:01:01


In [3]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=e20c66afed4ed82cb0e3b8f79816943e87b07d7dbe3248f188de2c53347f9101
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc


# Imports

In [6]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import *
from pylatexenc import *
import random
import numpy as np

# QKD

In [13]:
def QKD(n_qubits):
    backend = Aer.get_backend('qasm_simulator')

    # Choose randomly the bits and the bases for Alice
    alice_bits = [random.randint(0, 1) for _ in range(n_qubits)]
    alice_bases = [random.choice(['Z', 'X']) for _ in range(n_qubits)]

    # Choose randomly the measurement bases for Bob
    bob_bases = [random.choice(['Z', 'X']) for _ in range(n_qubits)]

    bob_results = []

    # For each qubit create a quantum circuit with one qubit and one classical bit
    for i in range(n_qubits):
        qc = QuantumCircuit(1, 1)

        # Alice
        if alice_bases[i] == 'Z':  # Z basis: bit 0 is |0> and bit 1 is |1>
            if alice_bits[i] == 1:  # transform to |1>
                qc.x(0)
        else:  # X basis: bit 0 is |+> and bit 1 is |->
            if alice_bits[i] == 0:
                qc.h(0)  # apply Hadamard gate to transform |0> to |+>
            else:
                qc.x(0)  # transform to |1>
                qc.h(0)  # apply Hadamard gate to transform |1> to |->

        # Apply Hadamard if Bob's base is X
        if bob_bases[i] == 'X':
            qc.h(0)

        # Apply measurement
        qc.measure(0, 0)

        qkd_transpiled = transpile(qc, backend)
        job = backend.run(qkd_transpiled)
        counts = job.result().get_counts()
        measured_bit = int(list(counts.keys())[0])
        bob_results.append(measured_bit)

        fig = qkd_transpiled.draw(output='mpl')
        fig.savefig("quantum_key_distribution_circuit_" + str(measured_bit) + ".png")

    # Compare Alice and Bob bases to keep only the bits where their bases match
    generated_key = []
    for i in range(n_qubits):
        if alice_bases[i] == bob_bases[i]:
            generated_key.append(alice_bits[i])
            print(f"Qubit {i}: Basis match: Alice's Basis({alice_bases[i]}), Bob's Basis({bob_bases[i]}). Alice's bit: {alice_bits[i]}, Bob's measurement: {bob_results[i]}")
        else:
            print(f"Qubit {i}: Basis mismatch: Alice's Basis({alice_bases[i]}), Bob's Basis({bob_bases[i]}). Alice's bit: {alice_bits[i]}, Bob's measurement: {bob_results[i]}")

    print("\nFinal key:", generated_key)


In [14]:
QKD(15)

Qubit 0: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 0, Bob's measurement: 0
Qubit 1: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 0, Bob's measurement: 0
Qubit 2: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 1, Bob's measurement: 1
Qubit 3: Basis mismatch: Alice's Basis(X), Bob's Basis(Z). Alice's bit: 1, Bob's measurement: 1
Qubit 4: Basis mismatch: Alice's Basis(X), Bob's Basis(Z). Alice's bit: 0, Bob's measurement: 1
Qubit 5: Basis mismatch: Alice's Basis(X), Bob's Basis(Z). Alice's bit: 1, Bob's measurement: 1
Qubit 6: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 0, Bob's measurement: 0
Qubit 7: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 1, Bob's measurement: 1
Qubit 8: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 0, Bob's measurement: 0
Qubit 9: Basis match: Alice's Basis(X), Bob's Basis(X). Alice's bit: 1, Bob's measurement: 1
Qubit 10: Basis mismatch: Alice's Basis(Z), Bob's Basis(X). A